# Creación de conversaciones HTTP

### Descripción: 
Con este código se va a intentar crear todos los pasos de una conversación HTTP, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificaciones

In [ ]:
import random
import re
import pickle

import openai
openai.api_key = "FILL"

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.inet import IP, TCP
from scapy.layers.http import *

openai.__version__

'1.10.0'

## Step 1: Generación de la descipción de una comunicación HTTP
---

<center><img src="https://notes.shichao.io/tcpv1/figure_13-1.png">
    
<center> Esta imagen es genérica, se intentará crear tanto la parte de hanshake como la terminación de la conversación en una primera aproximación. El intercambio de información siguiendo el protocolo HTTP se hace en la parte intermedia de la conversación, post handshake.


In [2]:
query_p = "This is a IPv4 TCP-HTTP conversation summary:\n"
query_p += 'Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"\n\n'

query_p += "x,y = random.sample(range(1, 100), 2)\n"
query_p += 'pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_1[TCP].payload)+1\n'
query_p += 'pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_2[TCP].payload)+1\n'
query_p += 'pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_3[TCP].payload)\n'

query_p += 'pkt_4= IP(src="193.24.227.230",  dst="9.9.9.9") / TCP(sport=40301, dport=80, flags="PA", seq=y, ack=x, window=8192) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webernetz.net", Path="/")\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_4[TCP].payload)\n'
query_p += 'pkt_5= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y, window=4128)\n\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_5[TCP].payload)\n'
query_p += 'pkt_6= IP(src="9.9.9.9",  dst="193.24.227.230") / TCP(sport=80, dport=40301, flags="PA", seq=x, ack=y, window=8192) / HTTP() / HTTPResponse(Http_Version= "HTTP/1.1", Status_Code= "200", Reason_Phrase= "OK", Content_Type="text/html") / "<HTML><BODY><H!>Hello World!</H1></BODY></HTML>"\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_6[TCP].payload)\n'
query_p += 'pkt_7= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_7[TCP].payload)\n'

query_p += 'pkt_8= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="FA", seq=y, ack=x, window=4128)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_8[TCP].payload)\n'
query_p += 'pkt_9= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y+1, window=256)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_9[TCP].payload)\n'
query_p += 'pkt_10= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="FA", seq=x, ack=y+1, window=256)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_10[TCP].payload)\n'
query_p += 'pkt_11= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y+1, ack=x+1, window=4128)\n\n'
query_p += 'y += len(pkt_11[TCP].payload)\n'

query_p += "Variable names in the HTTP definition are extremely important, use the exact same word. For example, use Http_Version, not http_version, use Method, not method.\n\n"
query_p += "In the HTTP response, if the variable code != 200, don´t use the Content_Type variable\n"
query = "Use the same dport for HTTPRequest (dport=80) and sport for HTTPResponse (sport=80), as this means we are creating a HTTP conection and using another port like 22 might make it seem like SSH traffic\n\n"
query += 'Pay special attention to the sequence and ack numbers so they make sense for a TCP conversation. The must be continuous throgh the conversation. Here you have a little explanation of how they work:\n'
query += 'The Sequence Number (SEQ) marks the starting byte of each segment sent. It increments by the number of bytes in the payload of each outgoing segment, reflecting the continuous flow of data.\n'
query += 'The Acknowledgment Number (ACK) acknowledges the receipt of data. It corresponds to the next expected byte the receiver anticipates from the sender. As segments arrive at the receiver, the ACK number adjusts to reflect the highest contiguous sequence number received.\n'
query += 'So, SEQ numbers increase with outgoing data, while ACK numbers advance with incoming acknowledgments, ensuring synchronized data exchange in a TCP connection.\n\n'

query += "Take your time (few seconds) to generate the commands and compare your proposed solution with real HTTP functioning, if anything is different, change it in order to do as real HTTP\n\n"

## Step 2: Carga de posibles conversaciones
---

In [11]:
with open("../../data/Conversations/HTTP/HTTP_Conv_Testing_v4.txt","r",encoding="utf8") as f:
    Conv_summaries = f.read().splitlines()
print("Number of conversation summaries: " + str(len(Conv_summaries)))

Number of conversation summaries: 161


### 2.1 Carga de posibles conversaciones anteriores para concatenarlas

In [12]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [13]:
try:
    responses = pickle.load( open("../../data/Conversations/HTTP/pickle/HTTP_validation_pairs_Aday_v3.pkl", "rb" ))
except:
    responses = []
    
print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 51


## Step 3: Generación de comandos de paquetes conversaciones TCP-HTTP
---

In [14]:
summaries_per_request = 1 # por que normalmente solo querremos generar una conversación

In [15]:
index = 0

fallos = 0
for i in range(len(Conv_summaries)):

    query1 = query_p
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")")
    

    ## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
    summary = Conv_summaries[i]
    text_sum = ""
    
    
    query1 += "\nThis is a conversation summary:\n"
    query1 += summary + "\n"
    text_sum += summary + "\n"
    query1 += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. Be sure you are doing 'time.sleep(abs(random.gauss(0, 0.03)))' between every pair of packets\n"

    #Query es el mensaje que le envias al modelo
    print(query1)

    #Esta es la petición que le mandas al modelo de IA
    completion = openai.completions.create(
        model="gpt-3.5-turbo-instruct", 
        prompt=query1,
        max_tokens=2600,
        temperature=0.1)

    
    print("...Generated!")
    
    try: 
        exec(completion.choices[0].text)
        pkt_list = [pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7, pkt_8, pkt_9, pkt_10, pkt_11]
    
        # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
        responses.append(response(text_sum, completion))
    
        # Guardamos los paquetes generados en un pcap
        with open("../../data/Conversations/HTTP/pcap/HTTP_Conv4_generated_3_5_turbo-instruct_conv_v3.pcap", "ba+") as f:
            wrpcap(f, pkt_list, append=True)

        # Guardamos tanto la petición como la respuesta en un archivo
        pickle.dump(responses, open( "../../data/Conversations/HTTP/pickle/HTTP_validation_pairs_Aday_v3.pkl", "wb" ) )
        
        del pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7, pkt_8, pkt_9, pkt_10, pkt_11, pkt_list
        
        i+=1
    except:
        fallos +=1
        
        if fallos <= 3:
            print("+++++++++++++++++++++++++ ERROR WHILE EXECUTING THE COMMANDS GENERATED. regenerating... +++++++++++++++++++++++++")
        else:
            print("+++++++++++++++++++++++++ ERROR WHILE EXECUTING THE COMMANDS GENERATED. too many errors, on to the next one... +++++++++++++++++++++++++")
            i+=1
        

    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 1)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)

time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_3[TCP].payload)
pkt_4= IP(src="193.24.227.230",  dst="9.9.9.9") / TCP(sport=40301, dpo

...Generated!
+++++++++++++++++++++++++ ERROR WHILE EXECUTING THE COMMANDS GENERATED. regenerating... +++++++++++++++++++++++++
Number of completitions done: 52
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 4)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y

...Generated!
+++++++++++++++++++++++++ ERROR WHILE EXECUTING THE COMMANDS GENERATED. too many errors, on to the next one... +++++++++++++++++++++++++
Number of completitions done: 52
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 7)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dpo

...Generated!
+++++++++++++++++++++++++ ERROR WHILE EXECUTING THE COMMANDS GENERATED. too many errors, on to the next one... +++++++++++++++++++++++++
Number of completitions done: 53
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 10)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dp

...Generated!
+++++++++++++++++++++++++ ERROR WHILE EXECUTING THE COMMANDS GENERATED. too many errors, on to the next one... +++++++++++++++++++++++++
Number of completitions done: 55
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 13)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dp

...

...Generated!
Number of completitions done: 127
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 148)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)

time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_3[TCP].payload)
pkt_4= IP(src="193.2

...Generated!
+++++++++++++++++++++++++ ERROR WHILE EXECUTING THE COMMANDS GENERATED. too many errors, on to the next one... +++++++++++++++++++++++++
Number of completitions done: 127
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 151)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, 

...Generated!
Number of completitions done: 129
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 154)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)

time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_3[TCP].payload)
pkt_4= IP(src="193.2

...Generated!
Number of completitions done: 132
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 157)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)

time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_3[TCP].payload)
pkt_4= IP(src="193.2

...Generated!
Number of completitions done: 133
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 160)
This is a IPv4 TCP-HTTP conversation summary:
Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"

x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)

time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_3[TCP].payload)
pkt_4= IP(src="193.2

## Step 4: Estudio de los resultados
---

In [256]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

Source: IP="192.168.0.100", port=8080, Window: 3768 // Destination: IP="192.168.0.101", port=80, Window: 6536 // Others: Host="http://www.twitter.com/", Path="", Code=200,  Reason_Phrase= "OK"


x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="192.168.0.100", dst="192.168.0.101")/TCP(sport=8080, dport=80, flags="S", seq=y, window=3768)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="192.168.0.101", dst="192.168.0.100")/TCP(sport=80, dport=8080, flags="SA", seq=x, ack=y, window=6536)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="192.168.0.100", dst="192.168.0.101")/TCP(sport=8080, dport=80, flags="A", seq=y, ack=x, window=3768)

time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_3[TCP].payload)
This part between the handshake and the teardown must be filled with bidirectional realistic HTTP traffic, here is an example of these packets:
pkt_4= IP(src="192.168.0.100",  dst="192.168.0.101") / TCP(sport=8080, d

### 4.1 Generación de paquetes

In [211]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
pkt_list = [pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7, pkt_8, pkt_9, pkt_10, pkt_11]
print(pkt_list)

[<IP  frag=0 proto=tcp src=10.0.0.200 dst=10.0.0.201 |<TCP  sport=9352 dport=http seq=99 flags=S window=8192 |>>, <IP  frag=0 proto=tcp src=10.0.0.201 dst=10.0.0.200 |<TCP  sport=http dport=9352 seq=6 ack=100 flags=SA window=1384 |>>, <IP  frag=0 proto=tcp src=10.0.0.200 dst=10.0.0.201 |<TCP  sport=9352 dport=http seq=100 ack=7 flags=A window=8192 |>>, <IP  frag=0 proto=tcp src=10.0.0.200 dst=10.0.0.201 |<TCP  sport=9352 dport=http seq=100 ack=7 flags=PA window=8192 |<HTTP  |<HTTPRequest  Method='GET' Path='/' Http_Version='HTTP/1.1' Host='ip.webernetz.net' |>>>>, <IP  frag=0 proto=tcp src=10.0.0.201 dst=10.0.0.200 |<TCP  sport=http dport=9352 seq=7 ack=142 flags=A window=1384 |>>, <IP  frag=0 proto=tcp src=10.0.0.201 dst=10.0.0.200 |<TCP  sport=http dport=9352 seq=7 ack=142 flags=PA window=8192 |<HTTP  |<HTTPResponse  Http_Version='HTTP/1.1' Status_Code='200' Reason_Phrase='OK' Content_Type='text/html' |<Raw  load='<HTML><BODY><H1>Hello World!</H1></BODY></HTML>' |>>>>>, <IP  frag=0 p

In [213]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 11 packets.


## Step 5: Introducción en un pcap
---

In [215]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Conversations/HTTP/pcap/HTTP_Conv2_generated_3_5_turbo-instruct_conv_ALL.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)